In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
!pip install pillow
from google.colab import drive
drive.mount('/content/drive')


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,775 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,508 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,161 k

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import os
import requests
from PIL import Image
from io import BytesIO
from urllib.parse import urlparse

In [ ]:
save_dir = "/content/drive/MyDrive/scraped_images_narender"
os.makedirs(save_dir, exist_ok=True)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

In [ ]:
def clean_url(url):
    return url.split("?")[0]

def scroll_and_collect(driver, num_needed, collected_urls):
    while len(collected_urls) < num_needed:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            srcset = img.get_attribute("srcset")
            if srcset:
                highest_res = srcset.strip().split(",")[-1].split()[0]
                cleaned = clean_url(highest_res)
                collected_urls.add(cleaned)
        print(f"Collected: {len(collected_urls)} URLs")
    return collected_urls

def download_images(urls, save_dir, batch_num, resize_dim=(256, 256)):
    count = 0
    for i, url in enumerate(urls):
        try:
            img_data = requests.get(url, timeout=10).content
            img = Image.open(BytesIO(img_data)).convert("RGB")
            img = img.resize(resize_dim)
            img.save(os.path.join(save_dir, f"batch{batch_num}___{i}.jpg"))
            count += 1
        except:
            pass
    print(f"Downloaded {count} images in batch {batch_num}")


In [ ]:
driver.get("https://unsplash.com")
total_images = 10000
batch_size = 2000

for batch in range(1, (total_images // batch_size) + 1):
    print(f"Starting batch {batch}")
    image_urls = set()
    image_urls = scroll_and_collect(driver, batch_size, image_urls)
    download_images(list(image_urls), save_dir, batch_num=batch)
    time.sleep(5)


Starting batch 1
Collected: 91 URLs
Collected: 109 URLs
Collected: 130 URLs
Collected: 144 URLs
Collected: 161 URLs
Collected: 180 URLs
Collected: 197 URLs
Collected: 218 URLs
Collected: 235 URLs
Collected: 253 URLs
Collected: 271 URLs
Collected: 286 URLs
Collected: 304 URLs
Collected: 339 URLs
Collected: 354 URLs
Collected: 370 URLs
Collected: 385 URLs
Collected: 398 URLs
Collected: 416 URLs
Collected: 433 URLs
Collected: 447 URLs
Collected: 464 URLs
Collected: 493 URLs
Collected: 507 URLs
Collected: 521 URLs
Collected: 535 URLs
Collected: 563 URLs
Collected: 577 URLs
Collected: 594 URLs
Collected: 609 URLs
Collected: 622 URLs
Collected: 634 URLs
Collected: 648 URLs
Collected: 665 URLs
Collected: 683 URLs
Collected: 696 URLs
Collected: 709 URLs
Collected: 722 URLs
Collected: 738 URLs
Collected: 752 URLs
Collected: 779 URLs
Collected: 807 URLs
Collected: 821 URLs
Collected: 835 URLs
Collected: 848 URLs
Collected: 863 URLs
Collected: 878 URLs
Collected: 892 URLs
Collected: 905 URLs
Coll

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (101756928 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (103833600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (111065900 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (102172140 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Downloaded 2006 images in batch 1
Starting batch 2
Collected: 2022 URLs
Downloaded 2022 images in batch 2
Starting batch 3
Collected: 2036 URLs
Downloaded 2036 images in batch 3
Starting batch 4
Collected: 2061 URLs
Downloaded 2061 images in batch 4
Starting batch 5
Collected: 2074 URLs
Downloaded 2074 images in batch 5


In [ ]:
print("Total images saved:", len(os.listdir(save_dir)))

Total images saved: 10267
